In [1]:
from bmvc_metrics import load_sol
from ctc_timings import get_im_centers, get_graph
from oracle import get_oracle, get_gt_graph
from tqdm import tqdm
from traccuracy.loaders import load_ctc_data
from visualize_lp_solution import load_tiff_frames

import json
import networkx as nx
import os
import pandas as pd
import time

In [2]:
ROOT_DATA_DIR = '/home/draga/PhD/data/cell_tracking_challenge/ST_Segmentations/'


In [3]:
ds_summary_df = pd.read_csv('/home/draga/PhD/data/cell_tracking_challenge/ST_Segmentations/ds_summary.csv', index_col=0)

In [5]:
with open('/home/draga/PhD/data/cell_tracking_challenge/ST_Segmentations/ctc_metrics.json', 'r') as f:
    metric_info = json.load(f)

In [ ]:
# used_ds_names = []
# used_seqs = []
# n_frames = []
# im_dim = []
# min_cells = []
# max_cells = []
# load_times = []

# for i, row in tqdm(enumerate(ds_summary_df.itertuples(), 1), desc='Building & Solving'):
#     ds_name = row.ds_name
#     seq = '{0:02}'.format(row.seq)
#     min_t = 0
#     max_t = row.n_frames - 1
#     corners = [(0, 0), eval(row.im_dim)]
#     key = f'{ds_name}_{row.seq}'
#     if isinstance(metric_info[key], dict):
#         print(f"Skipping {key}. Already computed metrics.")
#         continue
    
#     csv_pth = os.path.join(ROOT_DATA_DIR, f'{ds_name}_{seq}_coords.csv')
#     seg_pth = os.path.join(os.path.join(ROOT_DATA_DIR, ds_name), f'{seq}_ST/SEG/')
#     start_t = time.time()
#     coords, min_t, max_t, corners = get_im_centers(seg_pth)
#     duration = time.time() - start_t

#     used_ds_names.append(ds_name)
#     used_seqs.append(seq)        
#     n_frames.append(max_t - min_t + 1)
#     im_dim.append(corners[1])
#     n_cells_per_frame = coords.groupby('t').size()
#     min_cells.append(n_cells_per_frame.min())
#     max_cells.append(n_cells_per_frame.max())
#     load_times.append(duration)
#     coords.to_csv(csv_pth)

# ds_summary_df = pd.DataFrame({
#     'ds_name': ds_names,
#     'seq': seqs,
#     'n_frames': n_frames,
#     'im_dim': im_dim,
#     'min_cells': min_cells,
#     'max_cells': max_cells,
#     'load_time': load_times
# })

In [ ]:
# for i in range(len(used_ds_names)):
#     name = used_ds_names[i]
#     seq = used_seqs[i]
#     row_idx = ds_summary_df[(ds_summary_df.ds_name == name) & (ds_summary_df.seq == int(seq))].index[0]
#     ds_summary_df.at[row_idx,'load_time']=load_times[i]
#     ds_summary_df.at[row_idx, 'min_cells'] = min_cells[i]
#     ds_summary_df.at[row_idx, 'max_cells'] = max_cells[i]
    

In [ ]:
# ds_summary_df.to_csv('/home/draga/PhD/data/cell_tracking_challenge/ST_Segmentations/ds_summary.csv')

In [ ]:
# build and solve model for each sequence, starting with the smallest in number of frames
# ds_summary_df = ds_summary_df.sort_values(by='n_frames')
# ds_names_used = []
# ds_seqs_used = []
# build_times = []
# solve_times = []
# store_times = []
# for i, row in tqdm(enumerate(ds_summary_df.itertuples(), 1), desc='Building & Solving'):
#     ds_name = row.ds_name
#     seq = row.seq
#     min_t = 0
#     max_t = row.n_frames - 1
#     corners = [(0, 0), eval(row.im_dim)]
#     key = f'{ds_name}_{seq}'
#     if isinstance(metric_info[key], dict):
#         print(f"Skipping {key}. Already computed metrics.")
#         continue

#     print(f"Re-solving {key}.")
#     ds_seqs_used.append(seq)
#     ds_names_used.append(ds_name)
    
#     sol_dir = os.path.join(ROOT_DATA_DIR, ds_name, '{0:02}_RES/'.format(seq))
#     os.makedirs(sol_dir, exist_ok=True)
#     sol_pth = os.path.join(sol_dir, 'full_solution.graphml')

#     # read coords
#     coords_path = os.path.join(ROOT_DATA_DIR, '{0}_{1:02}_coords.csv'.format(ds_name, seq))
#     coords = pd.read_csv(coords_path)
    
#     # build graph
#     graph, build_time = get_graph(coords, min_t, max_t, corners)
#     build_times.append(build_time)

#     # solve model
#     m, flow = graph._to_gurobi_model()
#     m.Params.LogToConsole = 0
#     m.optimize()
#     if m.Status == 3:
#         infinite_cost_edges = graph._g.es.select(cost_ge=1e10)
#         graph._g.delete_edges(infinite_cost_edges)
#         m, flow = graph._to_gurobi_model()
#         m.optimize()
#         if m.Status != 2:
#             raise ValueError(f"Attempted to remove infinite cost edges but model for {key} was still not solved.")
#     solve_times.append(m.Runtime)
    
#     # store on graph
#     store_time = graph.store_solution(m)
#     store_times.append(store_time)
    
#     # save solution graph
#     nx_g = graph.convert_sol_igraph_to_nx()
#     nx.write_graphml_lxml(nx_g, sol_pth)

In [6]:
# for i in range(len(ds_names_used)):
#     name = ds_names_used[i]
#     seq = ds_seqs_used[i]
#     row_idx = ds_summary_df[(ds_summary_df.ds_name == name) & (ds_summary_df.seq == int(seq))].index[0]
#     ds_summary_df.at[row_idx,'build_time']= build_times[i]
#     ds_summary_df.at[row_idx, 'solve_time'] = solve_times[i]
#     ds_summary_df.at[row_idx, 'store_time'] = store_times[i]


In [12]:
# ds_summary_df.to_csv('/home/draga/PhD/data/cell_tracking_challenge/ST_Segmentations/ds_summary.csv')

In [ ]:
# merges = {}
# for i, row in enumerate(ds_summary_df.itertuples(), 1):
#     ds_name = row.ds_name
#     seq = row.seq
#     merges[f'{ds_name}_{seq}'] = {}
    
#     sol_dir = os.path.join(ROOT_DATA_DIR, ds_name, '{0:02}_RES/'.format(seq))
#     sol_pth = os.path.join(sol_dir, 'full_solution.graphml')
#     seg_pth = os.path.join(ROOT_DATA_DIR, ds_name, '{0:02}_ST/SEG/'.format(seq))
#     gt_pth = os.path.join(ROOT_DATA_DIR, ds_name, '{0:02}_GT/TRA/'.format(seq))
    
#     # load solution, check the number of merges
#     if not os.path.exists(sol_pth):
#         merges[f'{ds_name}_{seq}']['n_merges'] = -1
#         merges[f'{ds_name}_{seq}']['merge_nodes'] = []
#     else:
#         sol_data = load_sol(sol_pth, seg_pth)
#         sol = sol_data.tracking_graph.graph
#         oracle_node_df = pd.DataFrame.from_dict(sol.nodes, orient='index')
        
#         merge_nodes = [node for node in sol.nodes if len(sol.in_edges(node)) > 1]
#         merges[f'{ds_name}_{seq}']['n_merges'] = len(merge_nodes)
#         merges[f'{ds_name}_{seq}']['merge_nodes'] = merge_nodes
        
        


In [7]:
with open('/home/draga/PhD/data/cell_tracking_challenge/ST_Segmentations/merge_info.json', 'r') as f:
    merges = json.load(f)
    

In [ ]:
# oracles = {}
# ds_summary_df = ds_summary_df.sort_values(by='n_frames')
# oracle_pth = '/home/draga/PhD/data/cell_tracking_challenge/ST_Segmentations/oracles.json'
# for i, row in enumerate(ds_summary_df.itertuples(), 1):
#     ds_name = row.ds_name
#     seq = row.seq
#     key = f'{ds_name}_{seq}'

#     sol_dir = os.path.join(ROOT_DATA_DIR, ds_name, '{0:02}_RES/'.format(seq))
#     sol_pth = os.path.join(sol_dir, 'full_solution.graphml')
#     seg_pth = os.path.join(ROOT_DATA_DIR, ds_name, '{0:02}_ST/SEG/'.format(seq))
#     gt_pth = os.path.join(ROOT_DATA_DIR, ds_name, '{0:02}_GT/TRA/'.format(seq))

#     if merges[key]['n_merges'] and \
#         metric_info[key] is not None and \
#             metric_info[key]['DET'] < 1:
#         print(f"Running oracle for {key}")
#         sol_data = load_sol(sol_pth, seg_pth)
#         sol = sol_data.tracking_graph.graph
#         sol_ims = sol_data.segmentation
        
#         oracle_node_df = pd.DataFrame.from_dict(sol.nodes, orient='index')
#         merge_node_ids = merges[key]['merge_nodes']
#         gt_data = load_ctc_data(gt_pth)
#         gt_graph = gt_data.tracking_graph.graph
#         gt_ims = gt_data.segmentation
        
#         oracle = get_oracle(merge_node_ids, sol, gt_graph, sol_ims, gt_ims)
#         oracles[key] = oracle
#     else:
#         print(f"Skipping oracle for {key}")
#         oracles[key] = None
#     with open(oracle_pth, 'w') as f:
#         json.dump(oracles, f)

In [ ]:
with open('/home/draga/PhD/data/cell_tracking_challenge/ST_Segmentations/oracles.json', 'r') as f:
    oracles = json.load(f)

In [ ]:
actual_oracle_keys = [key for key in oracles if oracles[key] is not None]

In [ ]:
for key in actual_oracle_keys:
    oracle = oracles[key]
    introduce_vs = [oracle[key] for key in oracle if oracle[key]['decision'] == 'introduce']
    print(merges[key]['n_merges'], len(introduce_vs))
    print(introduce_vs)